In [5]:
import pandas as pd
import numpy as np


df = pd.read_csv('titanic.csv', index_col='PassengerId')
df

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Signing_date
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1911-05-17
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1911-07-23
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1911-09-08
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1911-06-26
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1911-10-25
...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,1911-08-17
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1911-08-07
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,1912-01-30


In [6]:
from sklearn.model_selection import train_test_split

X = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
y = df.Survived

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)



In [10]:
from sklearn.neural_network import MLPClassifier
from feature_engine.encoding import OneHotEncoder # Las redes neuronales funcionan mejor con OHE
from feature_engine.imputation import CategoricalImputer, MeanMedianImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from feature_engine.wrappers import SklearnTransformerWrapper

prep = Pipeline(steps=[
    ('cat_imp', CategoricalImputer(imputation_method='frequent')),
    ('num_imp', MeanMedianImputer(imputation_method='mean')),
    ('ohe', OneHotEncoder()),
    ('sc', SklearnTransformerWrapper(StandardScaler(), variables=['Fare', 'Age', 'Parch', 'SibSp']))
])

prep.fit_transform(X_train, y_train)


pipe = Pipeline(steps=[
    ('prep', prep),
    ('model', MLPClassifier(random_state=123,
                            max_iter=1000, # Se entrena de manera progresiva...
                            hidden_layer_sizes = (128, 64), # numero de neuronas por capa de neuronas...
                            activation = "relu" 
                            ))
])

# pipe.fit(X_train,y_train)

Pipeline(steps=[('prep',
                 Pipeline(steps=[('cat_imp',
                                  CategoricalImputer(imputation_method='frequent')),
                                 ('num_imp',
                                  MeanMedianImputer(imputation_method='mean')),
                                 ('ohe', OneHotEncoder()),
                                 ('sc',
                                  SklearnTransformerWrapper(transformer=StandardScaler(),
                                                            variables=['Fare',
                                                                       'Age',
                                                                       'Parch',
                                                                       'SibSp']))])),
                ('model',
                 MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=1000,
                               random_state=123))])

In [11]:
def train_function(pipe, X_train, X_test, y_train, y_test):
    pipe.fit(X_train, y_train)
    y_pred_train    = pipe.predict(X_train)
    y_pred          = pipe.predict(X_test)
    print('train')
    print(classification_report(y_train, y_pred_train, digits=4))
    print('test')
    print(classification_report(y_test, y_pred, digits=4))
    return pipe


train_function(pipe, X_train, X_test, y_train, y_test)

train
              precision    recall  f1-score   support

           0     0.8630    0.9472    0.9031       379
           1     0.9034    0.7664    0.8293       244

    accuracy                         0.8764       623
   macro avg     0.8832    0.8568    0.8662       623
weighted avg     0.8788    0.8764    0.8742       623

test
              precision    recall  f1-score   support

           0     0.8398    0.8941    0.8661       170
           1     0.7931    0.7041    0.7459        98

    accuracy                         0.8246       268
   macro avg     0.8164    0.7991    0.8060       268
weighted avg     0.8227    0.8246    0.8222       268



Pipeline(steps=[('prep',
                 Pipeline(steps=[('cat_imp',
                                  CategoricalImputer(imputation_method='frequent')),
                                 ('num_imp',
                                  MeanMedianImputer(imputation_method='mean')),
                                 ('ohe', OneHotEncoder()),
                                 ('sc',
                                  SklearnTransformerWrapper(transformer=StandardScaler(),
                                                            variables=['Fare',
                                                                       'Age',
                                                                       'Parch',
                                                                       'SibSp']))])),
                ('model',
                 MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=1000,
                               random_state=123))])

In [19]:
[i.shape for i in pipe[-1].coefs_]

[(10, 128), (128, 64), (64, 1)]